# Tokenizing

이름 :

기수 :

작성자 : 10 기 신재우

In [ ]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 시작 전 실행할 것들

In [ ]:
%pip install konlpy

In [ ]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pandas as pd
import numpy as np
import tqdm
import spacy
import torchtext
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import random
import os

### 경로 설정

In [ ]:
path_to_folder = '/content/drive/MyDrive/DSL/세션 준비/과제/[0225]RNN + Transformers_YourNameHere'

In [ ]:
path_to_dataset = os.path.join(path_to_folder, "Dataset/1_구어체(1).xlsx")

In [ ]:
dataset = pd.read_excel(path_to_dataset)
dataset = dataset[['원문', '번역문']]

In [ ]:
train_dataset = dataset.iloc[:30000, :]
val_dataset = dataset.iloc[30000:35000, :]
test_dataset = dataset.iloc[35000:40000, :]

kr_tokenizer = Kkma().morphs
en_tokenizer = spacy.load("en_core_web_sm").tokenizer

In [ ]:
for i in en_tokenizer("Hello, my name is Jae Woo Shin!"):
  print(i.text.lower())

hello
,
my
name
is
jae
woo
shin
!


In [ ]:
kr_tokenizer("안녕하세요 저는 신재우에요.")

['안녕', '하', '세요', '저', '는', '신', '재우', '에', '요', '.']

In [ ]:
for i in kr_tokenizer("안녕하세요 저는 신재우에요."):
  print(i)

안녕
하
세요
저
는
신
재우
에
요
.


In [ ]:
sos_token = "<sos>"
eos_token = "<eos>"

# 문제 1 :

아래 ___ 으로 되어 있는 코드를 채워주세요.

In [ ]:
def tokenize(row, en_tokenizer, kr_tokenizer, max_length, sos_token, eos_token):
  """
  여기에서의 목표는 Input 를 row 로 받아서 이것을 Tokenizer 를 통해서 벡터로 바꾸는 것이 목표입니다.

  row : 우리들의 input, Dataframe 형태로 되어 있으며 영어 텍스트는 row["번역문"], 한국어 텍스는 row["원문"] 으로 되어 있습니다.
  en_tokenizer : 영어 텍스트를 토큰으로 바꿔주는 토크나이저, 실행 방법은 en_tokenizer(en_text)
  kr_tokenizer : 한국어 텍스트를 토큰으로 바꿔주는 토크나이저, 실행 방법은 kr_tokenizer(kr_text)

  max_length : 1000 이며, 각 텍스트마다 토큰들이 이 개수를 넘기면 안되며, 이것을 넘는 토큰은 버린다
  sos_token : 시작 토큰, <sos>
  eos_token : 끝 토큰, <eos>

  return : en_tokens, kr_tokens, 각각 앞과 끝에 sos_token 과 eos_token 이 있어야 하는 각 텍스트를 토큰 형태로 바꿔준 리스트다.

  힌트 :
  - List Comprehension 을 통해서 텍스트에 있는 토큰들을 토큰화 시킨 것들을 리스트로 저장해줍니다.
  - 소문자로 바꾸기 위해서는 .lower() 함수를 사용해야 하며, 이것을 진행하기 전에 토큰들을 텍스트로 우선 바꿔줘야 합니다!
    사용 예: en_tokenizer(en_text).text.lower()
  """
  en_text = row["번역문"]
  kr_text = row["원문"]

  # 영어의 경우 모두 다 소문자로 바꿔주세요! 학습에 도움이 되기 위함입니다.

  en_tokens = ___
  kr_tokens = ___

  final_en_tokens = [sos_token] + en_tokens + [eos_token]
  final_kr_tokens = [sos_token] + kr_tokens + [eos_token]

  return pd.Series({"en_tokens": final_en_tokens, "kr_tokens": final_kr_tokens})

In [ ]:
from tqdm import tqdm
tqdm.pandas()

max_length = 1000

train_dataset[["en_tokens", "kr_tokens"]] = train_dataset.progress_apply(tokenize, axis=1, args=(en_tokenizer, kr_tokenizer, max_length, sos_token, eos_token))
val_dataset[["en_tokens", "kr_tokens"]] = val_dataset.progress_apply(tokenize, axis=1, args=(en_tokenizer, kr_tokenizer, max_length, sos_token, eos_token))
test_dataset[["en_tokens", "kr_tokens"]]= test_dataset.progress_apply(tokenize, axis=1, args=(en_tokenizer, kr_tokenizer, max_length, sos_token, eos_token))

### 위의 코드가 진행이 굉장히 오래되며, 이것을 오래 기다리기에는 힘들기에 저장을 시켜줘서 다음 파일인 Seq2Seq.ipynb 에서 진행하겠습니다

In [ ]:
path_train = os.path.join(path_to_folder, "Dataset/train_dataset_Tokenized.csv")
path_val = os.path.join(path_to_folder, "Dataset/val_dataset_Tokenized.csv")
path_test = os.path.join(path_to_folder, "Dataset/test_dataset_Tokenized.csv")

In [ ]:
train_dataset.to_csv(path_train)
val_dataset.to_csv(path_val)
test_dataset.to_csv(path_test)